# <center> Accuracy analysis with the Cramér-Rao bound </center>


In [ ]:
# %% load modules
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '.', '..'))

import numpy as np 

# to show maps of results 
import matplotlib.colors as mcol
import matplotlib.pyplot as plt
from model import theta, number_of_different_layers

In [ ]:
# paths
path_dataset = '***/TOAST/examples/data-horsehead/inputs'

FoV = np.load(f'{path_dataset}/FoV.npy', allow_pickle=True) # position-position-velocity cubes

Let $\theta$ the vector of unknowns such as $\theta = \{ log{\text{T}_{kin, i}},\, log{\text{n}_{H_2, i}},\, log{N_i},\, \text{FWHM}_i,\, C_{V, i}, ..., log{\text{T}_{kin, L}},\, log{\text{n}_{H_2, L}},\, log{N_L},\, \text{FWHM}_L,\, C_{V, L}\}$, where $L$ is the number of different layers composing the sight-line.


The Cramér-Rao bound $^{\star}$ (CRB) provides a lower bound of the variance on estimations for any
unbiased estimator, such as
$\text{var} (\widehat \theta_i) \geq [CRB(\theta)]_{ii} \text{ for } \widehat\theta_i \in {\widehat\theta}$.
Moreover, when for efficient estimator, one has 
$\text{var} (\widehat \theta_i) = [CRB(\theta)]_{ii} \text{ for } \widehat\theta_i \in {\widehat\theta}$.
We therefore compute the CRB on the estimation results to derive confidence intervals of reference. 

We provide a routine `launch_crb.py` to compute those error bars. 
In this notebook, we use the estimation results obtained in the example `analyze_map.ipynb`. 


**Note**: $^{\star}$ see Garthwaite, P. H., Jolliffe, I. T., & Jones, B. 1995, Statistical Inference (London:
Prentice Hall Europe)

To launch the routine, execute the cell below or type the shell command ```python launch_crb.py```. 
**Note**: For the 27 pixels in the field of view studied, this should take $\sim$ 5 minutes.

In [ ]:
# launch the computation of the CRBs 

os.system('python .././launch_crb.py')

### Result visualization 

**Note**: As the $log_{10}$ of $\text{T}_{kin}$, $\text{n}_{H_2}$ and ${N_i}$ are estimated, the precision are on the $log_{10}$ as well. The CRB on those variables can be read such as $\text{var} (\text{T}_{kin}) = \widehat{\text{T}_{kin}} * 10^{\pm \sqrt{CRB}}$. For instance, $CRB \sim 0.3$ corresponds to an error of a factor $\sim 2$. 

In [ ]:
# load CRB maps 
# vector theta : {log(Tkin), log(nH2), log(N(mol_ref)), FWHM, CV}, for each layer
maps_crb_rw = np.load(f'analyze_map/outputs/estimations/accuracies/maps_crb_rw.npy', allow_pickle = True)

# show maps 
titles = [
    r'log($T_{\mathrm{kin}}$ /K)',
    r'log($n_{\mathrm{H}_2}$ /cm$^{-3}$)',
    r'log($N$ /cm$^{-2}$)',
    r'FWHM [km/s]',
    r'$C_V$ [km/s]'
]

# for segmented colormap 
def create_colormap_CRB():
    bounds = np.array([0.01, 0.03, 0.06, 0.1, 0.3, 0.6, 1])
    norm = mcol.BoundaryNorm(boundaries=bounds, ncolors=256)
    cmap = mcol.LinearSegmentedColormap.from_list(
        "", ["green", "limegreen", "yellow", "orange", "red", "violet", "gray"])
    cmap.set_bad('white', 0.15)
    return cmap, norm
nicknames_layers = ['outer (foreground = background)', 'inner']
cmap, norm = create_colormap_CRB()

nrows, ncols = len(titles), number_of_different_layers
fig, axs = plt.subplots(nrows=nrows, ncols=ncols,
                                figsize=(8, 12), layout="constrained")

# compute the standard deviation = srqrt(CRB)
maps = [[] for i in range(number_of_different_layers)]
for u_idx in range(maps_crb_rw.shape[-1]):
    layer_idx = u_idx // len(theta)
    sqrt_map = np.ones(maps_crb_rw[:, :, u_idx, u_idx].shape)
    sqrt_map.fill(np.nan)
    sqrt_map = np.where(~np.isnan(maps_crb_rw[:, :, u_idx, u_idx]), np.sqrt(maps_crb_rw[:, :, u_idx, u_idx]), np.nan)
    sqrt_map = np.where(np.isnan(FoV), np.nan, sqrt_map)
    maps[layer_idx].append(sqrt_map)
    
for i in range(nrows):
    for j in range(ncols):
        ax = axs[i, j]

        if j == 0 : 
            ax.set_ylabel(f'{titles[i]}')
        if i == 0 : 
            ax.set_title(f'{nicknames_layers[j]}')

        map = maps[j][i]

        if i in [0, 1, 2] : 
            img = ax.imshow(
                map, 
                origin = 'lower', 
                cmap = cmap, # to comment to remove the segmented colormap
                norm = norm # to comment to remove the segmented colormap
            )
        else : 
            img = ax.imshow(
                map, 
                origin = 'lower'
            )    
        img = plt.colorbar(img, ax=ax)

fig.savefig(f'analyze_map/outputs/estimations/accuracies/maps_crb_rw.pdf')
